# Music Recommender System using Apache Spark and Python
**Estimated time: 8hrs**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this IPython file and do not clear any of the output you get from running your code.
* Upload this file onto moodle.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

## Necessary Package Imports

In [1]:
from pyspark.mllib.recommendation import *
import random
from operator import *

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [2]:
def checkInDic(item, alias_dict):
    if item in alias_dict:
        return alias_dict[item]
    else:
        return item    

def fixAliases(userArtistData, artistAlias):
    artist_alias_local = artistAlias.collect()
    alias_dict = dict()
    for entry in artist_alias_local:
        alias_dict[entry[0]] = entry[1]
    filtered = userArtistData.map(lambda l: (l[0], checkInDic(l[1], alias_dict), l[2]))
    return filtered

artistData = sc.textFile('artist_data_small.txt').map(lambda line: line.split("\t")).map(lambda row:(int(row[0]),row[1]))
artistAlias = sc.textFile('artist_alias_small.txt').map(lambda line: line.split("\t")).map(lambda row:(int(row[0]),int(row[1])))
userArtistData = sc.textFile('user_artist_data_small.txt').map(lambda line: line.split(" ")).map(lambda row:(int(row[0]),int(row[1]),int(row[2])))
userArtistData = fixAliases(userArtistData, artistAlias)

#print userArtistData.take(1)
#print artistAlias.take(1)
#print userArtistData.count()

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```


In [3]:
user_playCount = userArtistData.map(lambda x: (x[0],x[2])).combineByKey(
        (lambda value: (value,1)),
        (lambda tuple1, new_value: (tuple1[0]+new_value, tuple1[1]+1)),
        (lambda tuple1, tuple2: (tuple1[0]+tuple2[0], tuple1[1]+tuple2[1]) )
    )
#print user_playCount.take(1)

user_playCount_sorted = sorted(user_playCount.collect(), key=lambda x: x[1][0], reverse=True)
#print top_3[0], top_3[0][1][0]/top_3[0][1][1]

for i in range(3):
    print "User",user_playCount_sorted[i][0],"has a total play count of",user_playCount_sorted[i][1][0],"and a mean" \
    "play count of",user_playCount_sorted[i][1][0]/user_playCount_sorted[i][1][1]

User 1059637 has a total play count of 674412 and a meanplay count of 1878
User 2064012 has a total play count of 548427 and a meanplay count of 9455
User 2069337 has a total play count of 393515 and a meanplay count of 1519


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look as follows:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031
```

In [4]:
splits = userArtistData.randomSplit([2, 2, 1],seed=13)
trainData = splits[0]
validationData = splits[1]
testData = splits[2]

print trainData.take(3)
print validationData.take(3)
print testData.take(3)

print trainData.count()
print validationData.count()
print testData.count()

trainData.cache()
validationData.cache()
testData.cache()


[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000113, 5)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000112, 423)]
[(1059637, 1000094, 1), (1059637, 1000130, 19129), (1059637, 1000139, 4)]
19817
19633
10031


PythonRDD[20] at RDD at PythonRDD.scala:43

## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

In [5]:
def modelEval(model, test_data):
    test_user_set = set(test_data.map(itemgetter(0)).distinct().collect())
    all_artists_set = set(userArtistData.map(itemgetter(1)).distinct().collect())
    
    train_user_artists = trainData.map(lambda x:(x[0],x[1])).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()
    # (user,product,rating) -> (user,product) -> (user,[products])
    train_user_artists_mapping = dict((x[0],x[1]) for x in train_user_artists)

    test_user_artists = test_data.map(lambda x:(x[0],x[1])).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()
    test_user_artists_mapping = dict((x[0],x[1]) for x in test_user_artists)
    
    score = 0.0
    #print len(test_user_set)
    for user in test_user_set:
        actual_artists = set(test_user_artists_mapping[user])
        X = len(actual_artists)
        non_train_artists=all_artists_set-set(train_user_artists_mapping[user])
        # non_train_artists = all artists - artists in training set
        #   the rating for these non_train_artists are predicted by the predictAll()
        
        artist_list = []
        for artist in non_train_artists:
            artist_list.append((user,artist))
        artist_rdd=sc.parallelize(artist_list)
        # [(user,artist), ... ]
        
        predictions_rdd=model.predictAll(artist_rdd)
        # predict the rating for non_train_artists
        # returns [Rating(user=2069889, product=1000764, rating=-0.06997420544941978), ... ]
        #print predictions_rdd.take(1)
        
        predictions_list=predictions_rdd.map(lambda l: (l[1],l[2])).takeOrdered(X, key=lambda x: -x[1]) 
        # (user,product,rating) -> (product,rating) -> get top X (artist,rating) sorted by rating
        
        predictions_rdd=sc.parallelize(predictions_list)
        #print predictions_rdd.take(1)
        
        predicted_artists=predictions_rdd.map(lambda l:l[0]).collect()
        # top X(artist, rating) -> top X(artist)
        #print predicted_artists[0]
        
        intersection = set(predicted_artists) & actual_artists
        user_score = float(len(intersection)) / float(X)
        #print user,",",X,",",len(non_train_artists),",",len(intersection)
        score+=user_score
        
    return score/len(test_user_set)

#bestModel = ALS.trainImplicit(trainData, rank = 10, seed = 345)
#modelScore=modelEval(bestModel, validationData)
#print 'The model score for rank 10 is',modelScore

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is 0.090431
The model score for rank 10 is 0.095294
The model score for rank 20 is 0.090248
```

In [6]:
ranks = [2, 10, 20]
for rank in ranks:
    model = ALS.trainImplicit(trainData, rank = rank, seed = 345)
    model_score=modelEval(model, validationData)
    print 'The model score for rank',rank,'is',model_score

The model score for rank 2 is 0.0934617672415
The model score for rank 10 is 0.0978989450986
The model score for rank 20 is 0.0842589842136


Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.

In [8]:
best_model = ALS.trainImplicit(trainData, rank=10, seed=345)
modelEval(best_model, testData)

0.06124566051010857

## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: Brand New
Artist 1: Taking Back Sunday
Artist 2: Evanescence
Artist 3: Elliott Smith
Artist 4: blink-182
```

In [9]:
top_5 = best_model.recommendProducts(1059637, 5)

artistData_collected = artistData.collect()
artistID_name_map = dict()
for item in artistData_collected:
    artistID_name_map[item[0]] = item[1]
count = 0    
for item in top_5:
    print "Atrist " + str(count) + ": " + str(artistID_name_map[item.product])
    count += 1 

Atrist 0: blink-182
Atrist 1: Elliott Smith
Atrist 2: Taking Back Sunday
Atrist 3: Incubus
Atrist 4: Death Cab for Cutie
